In [1]:
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry
import requests
import random


from tqdm import tqdm
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

# MCLP를 위한 선형계획법 툴
from pulp import *

# 시각화 툴 : Pydeck
import pydeck as pdk

# 지리 데이터 전처리 툴 : Shapely
import shapely.speedups
shapely.speedups.enable()
from shapely.ops import unary_union
from shapely.geometry import Point, MultiLineString, mapping, shape

mapbox_key = 'pk.eyJ1IjoicWlxaTY1NCIsImEiOiJja2xnYW0xdWgyMmUyMnVxZWl2NGJpYng3In0.VIYWYBixklc-pAM9w7AjMA'

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


# 기존 cctv

In [2]:
cctv = pd.read_csv('data/1.김해시_CCTV설치현황.csv')
cctv.head()

,cctv_nm,type,install_ym,lon,lat
0,(동상동30)1_동상1092-31,방범용,'2020-11,128.883393,35.232738
1,(동상동31)1_동상1015-4,방범용,'2020-11,128.882718,35.236226
2,(동상동31)2_동상1015-4,방범용,'2020-11,128.882718,35.236226
3,(동상동31)3_동상1015-4,방범용,'2020-11,128.843626,35.229568
4,(동상동31)4_동상1015-4,방범용,'2020-11,128.883026,35.234773


# 추가 위치선정 cctv

In [3]:
cctv2 = pd.read_csv('final.csv',index_col=0) ##이름넣어!

In [5]:
# 격자매핑
grid = gpd.read_file('data/5.김해시_격자(100X100).geojson')

In [7]:
cctv2 = pd.merge(cctv2,grid,how='left',left_on='격자번호(100X100)',right_on='gid')

In [8]:
cctv2['coordinates'] = cctv2['geometry'].apply(lambda x : mapping(x)['coordinates'][0])
cctv2.head()

,우선순위,격자번호(100X100),gid,geometry,coordinates
0,1.0,마라243939,마라243939,"MULTIPOLYGON (((128.86600 35.23280, 128.86602 ...","(((128.86600366042447, 35.23279924257144), (12..."
1,2.0,마라278953,마라278953,"MULTIPOLYGON (((128.90467 35.24498, 128.90469 ...","(((128.9046746262055, 35.24497921902717), (128..."
2,3.0,마라258934,마라258934,"MULTIPOLYGON (((128.88241 35.22810, 128.88242 ...","(((128.88240729558154, 35.22810487367624), (12..."
3,4.0,마라243938,마라243938,"MULTIPOLYGON (((128.86599 35.23190, 128.86600 ...","(((128.86598854661403, 35.23189779005814), (12..."
4,5.0,마라276943,마라276943,"MULTIPOLYGON (((128.90232 35.23599, 128.90234 ...","(((128.90232176251328, 35.235990336473556), (1..."


# 시각화

In [24]:
kimhae = gpd.read_file('data/14.김해시_법정경계(읍면동).geojson')
kimhae['coordinates'] = kimhae['geometry'].apply(lambda x : mapping(x)['coordinates'][0])

# 김해 중심 위경도
center = [35.2415354, 128.8447851] 

base = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    kimhae, # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[255, 255, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    opacity = 0.005
)

view_state = pdk.ViewState( 
    latitude=center[0], 
    longitude=center[1], 
    zoom=11,
) 

In [27]:
scat = pdk.Layer('ScatterplotLayer',
                  cctv, 
                  get_position = ['lon','lat'], 
                  get_fill_color='[170, 249, 209]',
                  get_radius=50,
                  opacity = 0.1,
                  pickable=True, auto_highlight=True
                 ) 


layer = pdk.Layer('PolygonLayer', # 사용할 Layer 타입
                  cctv2, # 시각화에 쓰일 데이터프레임
                  get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
                  get_fill_color='[255, 0, 0, 200]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
                  opacity=0.8,
                  pickable=True, auto_highlight=True
)


r = pdk.Deck(layers=[base, scat, layer], initial_view_state=view_state,
             mapbox_key = 'pk.eyJ1IjoicWlxaTY1NCIsImEiOiJja2xnYW0xdWgyMmUyMnVxZWl2NGJpYng3In0.VIYWYBixklc-pAM9w7AjMA') 

In [28]:
r.to_html()